# Week3
## DCOPF 建模
根据《数据驱动最优潮流的误差可信量化及其抑制方法》式 2.2 至 2.5，DCOPF 具体建模如下

\begin{aligned}
\min_{P_G}\quad & C_G^{\!\top} P_G \qquad\text{(linear objective 线性目标)} \\ \text{s.t.}\quad & \mathbf{1}_G^{\!\top} P_G = \mathbf{1}_D^{\!\top} P_D \quad\text{(system balance 系统功率平衡)} \\ & P_L^{\min} \ \le\ M_G P_G + M_D P_D \ \le\ P_L^{\max} \quad\text{(line limits 线路约束)} \\ & P_G^{\min} \ \le\ P_G \ \le\ P_G^{\max} \quad\text{(generator bounds 机组约束).}
\end{aligned}
 

In [2]:
# pyright: reportAttributeAccessIssue=false, reportIndexIssue=false, reportOperatorIssue=false, reportGeneralTypeIssues=false, reportCallIssue=false, reportArgumentType=false, reportOptionalOperand=false, reportOptionalSubscript=false
import pyomo.environ as pyo
from typing import Any

def build_dc_opf_model() -> Any:
    """Abstract DC-OPF model (variables: P_G)
    Obj: minimize generation cost
    Constraints: system balance, line limits and generator bounds.
    """
    m: Any = pyo.AbstractModel()

    # --------- Sets ---------
    m.G = pyo.Set(doc="Generators")
    m.D = pyo.Set(doc="Demands")
    m.L = pyo.Set(doc="Lines") 

    # --------- Parameters ---------
    # Linear generation cost coefficients
    m.c2 = pyo.Param(m.G, within=pyo.Reals, doc="Linear cost coefficient C_G")
    m.c1 = pyo.Param(m.G, within=pyo.Reals, doc="Linear cost coefficient")
    m.c0 = pyo.Param(m.G, within=pyo.Reals, doc="Constant cost coefficient")

    # Demand vector (indexed by m.D)
    m.Pd = pyo.Param(m.D, doc="Demand at each demand node (fixed)")

    # Line flow limits
    m.P_L_max = pyo.Param(m.L, within=pyo.Reals, doc="Line flow upper limit")
    m.P_L_min = pyo.Param(m.L, within=pyo.Reals, doc="Line flow lower limit")

    # PTDFs: contribution of each generator / demand to line flows
    m.M_G = pyo.Param(m.L, m.G, within=pyo.Reals, doc="PTDFs for generators")
    m.M_D = pyo.Param(m.L, m.D, within=pyo.Reals, doc="PTDFs for demands")

    # Generator output bounds
    m.P_G_max = pyo.Param(m.G, within=pyo.Reals, doc="Generator max output")
    m.P_G_min = pyo.Param(m.G, within=pyo.Reals, doc="Generator min output")

    # --------- Variables ---------
    def _P_G_bounds(m: Any, g):
        return (m.P_G_min[g], m.P_G_max[g])
    m.P_G = pyo.Var(m.G, bounds=_P_G_bounds, doc="Generator outputs (MW)")

    # --------- Objective ---------
    def _gen_cost_rule(m: Any):
        return sum(m.c2[g] * m.P_G[g]**2 + m.c1[g] * m.P_G[g] + m.c0[g] for g in m.G)
    m.Obj = pyo.Objective(rule=_gen_cost_rule, sense=pyo.minimize)

    # --------- Constraints ---------
    # System balance: sum_g P_G = sum_d P_d
    def _system_balance_rule(m: Any):
        return sum(m.P_G[g] for g in m.G) == sum(m.Pd[d] for d in m.D)
    m.System_Balance = pyo.Constraint(rule=_system_balance_rule)

     # Line flow expression: f_l = M_G*P_G + M_D*P_d
    def _flow_def(m: Any, l):
        return sum(m.M_G[l, g]*m.P_G[g] for g in m.G) \
             + sum(m.M_D[l, d]*m.Pd[d] for d in m.D)
    m.Flow = pyo.Expression(m.L, rule=_flow_def)

    # Line limits: P_L_min <= f_l <= P_L_max
    def _flow_limits(m: Any, l):
        return pyo.inequality(m.P_L_min[l], m.Flow[l], m.P_L_max[l])
    m.LineLimits = pyo.Constraint(m.L, rule=_flow_limits)

    return m


In [3]:
# pyright: reportAttributeAccessIssue=false, reportIndexIssue=false, reportOperatorIssue=false, reportGeneralTypeIssues=false, reportCallIssue=false, reportArgumentType=false, reportOptionalOperand=false, reportOptionalSubscript=false
import numpy as np
import pyomo.environ as pyo
from typing import Any

# ---- helpers to build PTDF & Pyomo data from a PYPOWER case ----

def _bus_index_maps(ppc):
    bus = ppc["bus"]
    # PYPOWER uses 1-based bus numbers in the data; build map -> 0-based row indices
    bus_nums = bus[:, 0].astype(int)
    idx_of_busnum = {int(bn): i for i, bn in enumerate(bus_nums)}
    return bus_nums, idx_of_busnum

def _choose_slack(ppc, idx_of_busnum):
    # type column is 2nd field (index 1): 3=REF, 2=PV, 1=PQ
    REF = 3
    bus = ppc["bus"]
    ref_rows = np.where(bus[:,1] == REF)[0]
    if len(ref_rows) == 0:
        # fallback: smallest bus number
        return 0
    return int(ref_rows[0])

def _build_dc_matrices(ppc, slack_row):
    """Return (Bbus, Cf, b_line) with tap ratios accounted for, angles ignored."""
    bus = ppc["bus"]
    branch = ppc["branch"]

    nb = bus.shape[0]
    nl = branch.shape[0]

    # columns in branch: fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
    FBUS, TBUS, R, X, B, RATEA, RATEB, RATEC, RATIO, ANGLE, STATUS, ANGMIN, ANGMAX = range(13)

    # active lines
    on = branch[:, STATUS] > 0.5
    br = branch[on, :]

    # susceptance per branch (p.u. on system base); DC uses 1/x, include tap ratio
    x = br[:, X]
    if np.any(np.isclose(x, 0.0)):
        raise ValueError("Branch reactance x=0 found; DC PTDF undefined for that branch.")
    tap = br[:, RATIO].copy()
    tap[ (tap==0) | np.isnan(tap) ] = 1.0  # default tap=1
    b_series = 1.0 / x
    b_line = b_series / tap  # effective susceptance on "from" side

    # incidence matrix Cf (nl_on x nb): +1 at from, -1 at to
    fbus = br[:, FBUS].astype(int)
    tbus = br[:, TBUS].astype(int)

    # map bus numbers -> row indices
    bus_nums, idx_map = _bus_index_maps(ppc)
    nl_on = br.shape[0]
    Cf = np.zeros((nl_on, nb))
    for k in range(nl_on):
        i = idx_map[int(fbus[k])]
        j = idx_map[int(tbus[k])]
        Cf[k, i] =  1.0 / tap[k]   # tap on from-side
        Cf[k, j] = -1.0

    # Bf and Bbus
    Bf   = (b_line[:, None] * Cf)                   # f = Bf * theta
    Bbus = Cf.T @ (b_line[:, None] * Cf)            # p = Bbus * theta

    # Reduce by removing slack row/col for inversion
    keep = [i for i in range(nb) if i != slack_row]
    Bbus_red = Bbus[np.ix_(keep, keep)]

    return Bbus, Bbus_red, Bf, Cf, b_line, keep

def _compute_ptdf(ppc):
    """Compute PTDF (lines × buses) w.r.t. chosen slack."""
    _, idx_map = _bus_index_maps(ppc)
    slack_row = _choose_slack(ppc, idx_map)
    Bbus, Bbus_red, Bf, Cf, _, keep = _build_dc_matrices(ppc, slack_row)

    # PTDF via columns: for each bus k, inject +1 at k and -1 at slack
    nb = Bbus.shape[0]
    nl = Bf.shape[0]
    H = np.zeros((nl, nb))  # PTDF[:, bus]

    # Pre-factorize reduced Bbus for speed
    # (Cholesky may fail if singular; use np.linalg.solve)
    for k in range(nb):
        if k == slack_row:
            # by definition, a 1 p.u. injection at slack balanced at slack gives zero net — column zeros
            H[:, k] = 0.0
            continue
        # build p_red: +1 at k, -1 at slack -> in reduced system it's +1 at k, slack removed
        p = np.zeros(nb)
        p[k] =  1.0
        p[slack_row] = -1.0
        p_red = p[keep]
        theta_red = np.linalg.solve(Bbus_red, p_red)

        # reconstruct full theta (insert 0 at slack position)
        theta = np.zeros(nb)
        theta[keep] = theta_red
        theta[slack_row] = 0.0

        # line flows for that unit transaction
        f = Bf @ theta
        H[:, k] = f

    # Only rows for in-service branches were built; map them back to full list of lines if needed
    return H, slack_row

def ppc_to_pyomo_data(ppc):
    """Produce sets & params for the PTDF-reduced Pyomo model you defined."""
    baseMVA = ppc["baseMVA"]
    bus = ppc["bus"]; gen = ppc["gen"]; branch = ppc["branch"]; gencost = ppc["gencost"]

    bus_nums, idx_map = _bus_index_maps(ppc)
    nb = bus.shape[0]; ng = gen.shape[0]; nl = branch.shape[0]

    # Sets
    # Generators: index by 0..ng-1
    G = [f"g{g}" for g in range(ng)]
    # Demands: use buses with Pd>0 as demand points
    PD_COL = 2  # Pd column in bus
    load_rows = [i for i in range(nb) if bus[i, PD_COL] > 1e-9]
    D = [f"d{bus_nums[i]}" for i in load_rows]  # label by bus number
    # Lines: 0..nl-1
    L = [f"l{k}" for k in range(nl)]

    # Generator bounds & mapping gen->bus
    PMAX, PMIN, GBUS = 8, 9, 0
    Pg_max = {G[g]: float(gen[g, PMAX]) for g in range(ng)}
    Pg_min = {G[g]: float(gen[g, PMIN]) for g in range(ng)}
    g_bus_rows = [ idx_map[int(gen[g, GBUS])] for g in range(ng) ]

    # Costs (type-2 quadratic): columns are [2 startup shutdown n c2 c1 c0]
    c2 = {}; c1 = {}; c0 = {}
    for g in range(ng):
        c2[G[g]] = float(gencost[g, 4])
        c1[G[g]] = float(gencost[g, 5])
        c0[G[g]] = float(gencost[g, 6])

    # Demand vector
    Pd = { f"d{bus_nums[i]}": float(bus[i, PD_COL]) for i in load_rows }

    # Line limits (rateA). If zero, set a big number or compute from x; here use big-M.
    RATEA, STATUS = 5, 10
    bigM = 1e6
    P_L_max = {}
    P_L_min = {}
    active_mask = branch[:, STATUS] > 0.5
    for k in range(nl):
        if active_mask[k] and branch[k, RATEA] > 0:
            lim = float(branch[k, RATEA])  # MW approx (MVA on base)
        else:
            lim = bigM
        P_L_max[f"l{k}"] = lim
        P_L_min[f"l{k}"] = -lim

    # PTDF matrix (lines_in_service × buses)
    H, slack_row = _compute_ptdf(ppc)

    # Map PTDF rows back to all lines (including out-of-service as zeros)
    on = np.where(active_mask)[0]
    H_full = np.zeros((nl, nb))
    H_full[on, :] = H

    # Build M_G and M_D
    # For a generator at bus b_g (row index), column is H[:, b_g]
    # For a demand at bus b_d with Pd>0, the injection is NEGATIVE, so M_D = -H[:, b_d]
    M_G = {}
    for g in range(ng):
        col = H_full[:, g_bus_rows[g]]
        for k in range(nl):
            M_G[(f"l{k}", G[g])] = float(col[k])

    M_D = {}
    for i in load_rows:
        dkey = f"d{bus_nums[i]}"
        col = H_full[:, i]
        for k in range(nl):
            M_D[(f"l{k}", dkey)] = float(-col[k])  # minus sign for loads

    # Package for Pyomo AbstractModel DataPortal
    data = {
        "G": G,
        "D": D,
        "L": L,
        "Pg_min": Pg_min,
        "Pg_max": Pg_max,
        "c2": c2, "c1": c1, "c0": c0,
        "Pd": Pd,
        "P_L_min": P_L_min, "P_L_max": P_L_max,
        "M_G": M_G, "M_D": M_D,
        "meta": {"slack_row": int(slack_row), "baseMVA": float(baseMVA)}
    }
    return data

def create_instance_from_ppc(model, ppc) -> tuple[Any, dict]:
    """Build a ConcreteModel instance from ppc, avoiding DataPortal issues."""
    dat = ppc_to_pyomo_data(ppc)

    inst: Any = pyo.ConcreteModel()
    # Sets
    inst.G = pyo.Set(initialize=dat['G'])
    inst.D = pyo.Set(initialize=dat['D'])
    inst.L = pyo.Set(initialize=dat['L'])

    # Params
    inst.P_G_min = pyo.Param(inst.G, initialize=dat['Pg_min'])
    inst.P_G_max = pyo.Param(inst.G, initialize=dat['Pg_max'])
    inst.c2 = pyo.Param(inst.G, initialize=dat['c2'])
    inst.c1 = pyo.Param(inst.G, initialize=dat['c1'])
    inst.c0 = pyo.Param(inst.G, initialize=dat['c0'])
    inst.Pd = pyo.Param(inst.D, initialize=dat['Pd'])
    inst.P_L_min = pyo.Param(inst.L, initialize=dat['P_L_min'])
    inst.P_L_max = pyo.Param(inst.L, initialize=dat['P_L_max'])

    # M_G and M_D: provide 2-D Param initializers
    def mg_init(m: Any, l, g):
        return dat['M_G'].get((l, g), 0.0)
    def md_init(m: Any, l, d):
        return dat['M_D'].get((l, d), 0.0)
    inst.M_G = pyo.Param(inst.L, inst.G, initialize=mg_init, default=0.0)
    inst.M_D = pyo.Param(inst.L, inst.D, initialize=md_init, default=0.0)

    # Variables
    def _P_G_bounds(m: Any, g):
        return (inst.P_G_min[g], inst.P_G_max[g])
    inst.P_G = pyo.Var(inst.G, bounds=_P_G_bounds)

    # Objective
    def _gen_cost_rule(m: Any):
        return sum(inst.c2[g] * inst.P_G[g]**2 + inst.c1[g] * inst.P_G[g] + inst.c0[g] for g in inst.G)
    inst.Obj = pyo.Objective(rule=_gen_cost_rule, sense=pyo.minimize)

    # System balance
    def _system_balance_rule(m: Any):
        return sum(inst.P_G[g] for g in inst.G) == sum(inst.Pd[d] for d in inst.D)
    inst.System_Balance = pyo.Constraint(rule=_system_balance_rule)

    # Flow expression
    def _flow_def(m: Any, l):
        return sum(inst.M_G[l, g] * inst.P_G[g] for g in inst.G) + sum(inst.M_D[l, d] * inst.Pd[d] for d in inst.D)
    inst.Flow = pyo.Expression(inst.L, rule=_flow_def)

    # Line limits
    def _flow_limits(m: Any, l):
        return pyo.inequality(inst.P_L_min[l], inst.Flow[l], inst.P_L_max[l])
    inst.LineLimits = pyo.Constraint(inst.L, rule=_flow_limits)

    return inst, dat['meta']

In [4]:
# pyright: reportOptionalOperand=false, reportCallIssue=false, reportArgumentType=false, reportOptionalSubscript=false
from typing import Optional

# Load case and solve DCOPF
from pypower.api import case118

ppc = case118()
model = build_dc_opf_model()
instance, meta = create_instance_from_ppc(model, ppc)

# Solve with Gurobi
opt = pyo.SolverFactory('gurobi')
res = opt.solve(instance, tee=False)

print('Solver termination:', res.solver.termination_condition)
print('Objective value: ${:,.2f}/hr'.format(pyo.value(instance.Obj)))

# Display generator dispatch results
print(f'\nGenerator dispatch (showing active generators):')
print(f'{"Generator":<12} {"Output (MW)":>12}')
print('-' * 26)

active_count = 0
for g in sorted(list(instance.G), key=lambda x: int(str(x)[1:])):
    Pg_val_raw: Optional[float] = pyo.value(instance.P_G[g])
    if Pg_val_raw is None:
        continue
    Pg_val: float = float(Pg_val_raw)
    # Only show generators with non-trivial output
    if Pg_val > 0.01:
        print(f'{g:<12} {Pg_val:12.2f}')
        active_count += 1

print('-' * 26)
nG = len(list(instance.G))
total_generation: float = sum(float(pyo.value(instance.P_G[g]) or 0.0) for g in instance.G)
print(f'Active generators: {active_count} of {nG}')
print(f'Total generation: {total_generation:,.2f} MW')


Solver termination: optimal
Objective value: $125,947.87/hr

Generator dispatch (showing active generators):
Generator     Output (MW)
--------------------------
g4                 436.08
g5                  82.37
g10                213.20
g11                304.29
g13                  6.78
g19                 18.41
g20                197.69
g21                 46.52
g24                150.21
g25                155.05
g27                378.91
g28                379.87
g29                500.43
g36                462.24
g38                  3.88
g39                588.22
g44                244.21
g45                 38.76
g50                 34.89
--------------------------
Active generators: 19 of 54
Total generation: 4,242.00 MW


In [5]:
# pyright: reportOptionalSubscript=false, reportArgumentType=false
from typing import Any, Iterable

# --- Run DCOPF using PYPOWER built-in solver -----------------
from pypower.api import case118, rundcopf, printpf

# Load the case
ppc = case118()

# Run DC-OPF
results = rundcopf(ppc)

# Print summary
printpf(results)

# ---- Print generator outputs ----
print("\nGenerator dispatch results (MW):")
# Safely access generator matrix from results without ambiguous truth checks
gens_val = results.get("gen") if isinstance(results, dict) else None
if gens_val is None:
    gens_iter: Iterable[Any] = []
else:
    gens_iter = gens_val

# Column 0 = bus number, 1 = Pg, 8 = Pmax in MATPOWER/PYPOWER format
for i, row in enumerate(gens_iter, start=1):
    bus_num = int(row[0])
    Pg = row[1]
    Pmax = row[8]
    print(f"  Gen {i:02d} at Bus {bus_num:3d}:  Pg = {Pg:10.4f} MW   (Pmax = {Pmax:10.4f})")

PYPOWER Version 5.1.18, 10-Apr-2025 -- DC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.04 seconds
Objective Function Value = 125947.87 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses            118     Total Gen Capacity    9966.2           0.0 to 0.0
Generators        54     On-line Capacity      9966.2           0.0 to 0.0
Committed Gens    54     Generation (actual)   4242.0               0.0
Loads             99     Load                  4242.0               0.0
  Fixed           99       Fixed               4242.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches         186     Losses (I^2 * Z)         0